In [12]:
from pyspark.sql import functions as F
from helper import run_forecast, aggregate_to_granularity, build_features, train_test_split

from pyspark.sql import SparkSession

# Give Spark way more memory since you have 32GB RAM available
spark = SparkSession.builder \
    .appName("TimeSeriesForecast") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Load data (must include columns: date, sales, family, store_nbr)
df = (   
      spark.read.parquet('../notebooks/data/train.parquet')

      .withColumn("date", F.to_date(F.col("date"))))



In [13]:
df.show(2)

+----+------+------+--------------------+----------+---------+-----+-----------+-------+----------+----------+--------------------+----------------+--------------+---------------+--------------+-----------+------------+-----+----+-------------+
|type| state|  city|              family|      date|store_nbr|sales|onpromotion|cluster|is_holiday|dcoilwtico|       hash_features|strIndxer_family|strIndxer_city|strIndxer_state|strIndxer_type|day_of_week|day_of_month|month|year|is_salary_day|
+----+------+------+--------------------+----------+---------+-----+-----------+-------+----------+----------+--------------------+----------------+--------------+---------------+--------------+-----------+------------+-----+----+-------------+
|   D| Azuay|Cuenca|SCHOOL AND OFFICE...|2013-01-01|       37|  0.0|          0|      2|         1|     93.14|(1024,[124,560,79...|              27|            14|             15|             4|          3|           1|    1|2013|            0|
|   C|Guayas|Playas|

In [14]:
# agg_cols = ['family', 'store_nbr', 'state', 'city', 'type', 'cluster'] # there is one store per city and type and cluster so adding those feature is redundant
# agg_cols = ['family', 'store_nbr'] # there is one store per city and type and cluster so adding those feature is redundant

# # agg_cols = ['family']

# df_agg = aggregate_to_granularity(
#     df=df,
#     date_col='date',
#     target_col='sales',
#     group_cols=agg_cols,
#     agg="sum",  # sales are additive
#     # extra_numeric_aggs={"dcoilwtico": "mean"}  # optional
# )
# print(df_agg.count())
# df_agg.show(2)

# df_feat = build_features(
#     df=df_agg,
#     date_col='date',
#     target_col='sales',
#     group_cols=agg_cols,
#     lags=[1, 7, 14, 28],
#     mas=[7, 28],
#     # add_time_signals=cfg["features"]["add_time_signals"]
# )
# df_feat.show(10)
# print(df_feat.count())
# # df_feat.sort(agg_cols + ["date"]).select(agg_cols + ['date', 'sales', 'dcoilwtico', 'lag_1', 'lag_2']).show(10)

In [ ]:
df.sample(0.1)

+----+------+------+--------------------+----------+---------+-----+-----------+-------+----------+----------+--------------------+----------------+--------------+---------------+--------------+-----------+------------+-----+----+-------------+
|type| state|  city|              family|      date|store_nbr|sales|onpromotion|cluster|is_holiday|dcoilwtico|       hash_features|strIndxer_family|strIndxer_city|strIndxer_state|strIndxer_type|day_of_week|day_of_month|month|year|is_salary_day|
+----+------+------+--------------------+----------+---------+-----+-----------+-------+----------+----------+--------------------+----------------+--------------+---------------+--------------+-----------+------------+-----+----+-------------+
|   C|Guayas|Playas|SCHOOL AND OFFICE...|2013-01-02|       35|  0.0|          0|      3|         0|     93.14|(1024,[191,373,47...|              27|             9|              7|             1|          4|           2|    1|2013|            0|
|   D|Manabi| Manta|

In [18]:
# Load config from YAML or dict
import yaml, json
cfg = yaml.safe_load(open("forecast_config.yaml"))

out = run_forecast(df.sample(0.1), cfg)

display(out["predictions"])         # per-group predictions on test window

ic| helper.py:488 in run_forecast() at 23:49:36.106


+----------+---------+----------+-----+-----+-----+------+------+-----------------+-----------------+---+---+----------+-----+----+
|    family|store_nbr|      date|sales|lag_1|lag_7|lag_14|lag_28|             ma_7|            ma_28|dow|dom|weekofyear|month|year|
+----------+---------+----------+-----+-----+-----+------+------+-----------------+-----------------+---+---+----------+-----+----+
|AUTOMOTIVE|        6|2013-12-23|  7.0|  4.0|  6.0|   1.0|   4.0|6.428571428571429|4.357142857142857|  2| 23|        52|   12|2013|
|AUTOMOTIVE|        6|2014-01-17|  3.0|  7.0| 10.0|   2.0|   6.0|5.428571428571429|             4.25|  6| 17|         3|    1|2014|
+----------+---------+----------+-----+-----+-----+------+------+-----------------+-----------------+---+---+----------+-----+----+
only showing top 2 rows


ic| ): None


+----------+---------+----------+-----+-----+-----+------+------+----+-----------------+---+---+----------+-----+----+
|    family|store_nbr|      date|sales|lag_1|lag_7|lag_14|lag_28|ma_7|            ma_28|dow|dom|weekofyear|month|year|
+----------+---------+----------+-----+-----+-----+------+------+----+-----------------+---+---+----------+-----+----+
|AUTOMOTIVE|       10|2013-09-07|  4.0|  3.0|  3.0|   2.0|   3.0| 3.0|             3.25|  7|  7|        36|    9|2013|
|AUTOMOTIVE|       10|2013-10-01|  3.0|  4.0|  3.0|   3.0|   6.0| 3.0|3.142857142857143|  3|  1|        40|   10|2013|
+----------+---------+----------+-----+-----+-----+------+------+----+-----------------+---+---+----------+-----+----+
only showing top 2 rows


ic| else:: None


+----------+---------+----------+-----+-----+-----+------+------+-----------------+------------------+---+---+----------+-----+----+
|    family|store_nbr|      date|sales|lag_1|lag_7|lag_14|lag_28|             ma_7|             ma_28|dow|dom|weekofyear|month|year|
+----------+---------+----------+-----+-----+-----+------+------+-----------------+------------------+---+---+----------+-----+----+
|AUTOMOTIVE|       10|2016-10-25|  3.0|  2.0|  6.0|   3.0|   4.0|              2.0|3.3214285714285716|  3| 25|        43|   10|2016|
|AUTOMOTIVE|       10|2016-11-02|  3.0|  3.0|  2.0|   1.0|   3.0|2.142857142857143|3.3214285714285716|  4|  2|        44|   11|2016|
+----------+---------+----------+-----+-----+-----+------+------+-----------------+------------------+---+---+----------+-----+----+
only showing top 2 rows


ic| est = m: None
ic| ac: GBTRegressor_57cf7b8a6b6d
ic| ------------: ['lag_1',
                   'lag_7',
                   'lag_14',
                   'lag_28',
                   'ma_7',
                   'ma_28',
                   'dow',
                   'dom',
                   'weekofyear',
                   'month',
                   'year']


+-----+-----+------+------+----+-----------------+---+---+----------+-----+----+
|lag_1|lag_7|lag_14|lag_28|ma_7|            ma_28|dow|dom|weekofyear|month|year|
+-----+-----+------+------+----+-----------------+---+---+----------+-----+----+
|  3.0|  3.0|   2.0|   3.0| 3.0|             3.25|  7|  7|        36|    9|2013|
|  4.0|  3.0|   3.0|   6.0| 3.0|3.142857142857143|  3|  1|        40|   10|2013|
+-----+-----+------+------+----+-----------------+---+---+----------+-----+----+
only showing top 2 rows


ic| tration: train → forecast → evalua: None
ic| ecast(df: DataF: 'sales'
ic| ["data"]
    s: ['family', 'store_nbr']
ic| ["split"]: ['lag_1',
                'lag_7',
                'lag_14',
                'lag_28',
                'ma_7',
                'ma_28',
                'dow',
                'dom',
                'weekofyear',
                'month',
                'year']
ic| ["m: GBTRegressor_57cf7b8a6b6d
ic| indexed_cols: ['family_ohe']
ic| feature_cols: ['lag_1',
                   'lag_7',
                   'lag_14',
                   'lag_28',
                   'ma_7',
                   'ma_28',
                   'dow',
                   'dom',
                   'weekofyear',
                   'month',
                   'year']
ic| indexed_cols: ['family_ohe', 'store_nbr_ohe']
ic| feature_cols: ['lag_1',
                   'lag_7',
                   'lag_14',
                   'lag_28',
                   'ma_7',
                   'ma_28',
          

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `s` cannot be resolved. Did you mean one of the following? [`y`, `date`, `family`, `store_nbr`, `prediction`]. SQLSTATE: 42703;
'Project [family#1920, store_nbr#1922, date#2091, y#2894, prediction#2890, lag(prediction#2890, -7, null) windowspecdefinition('s, 't, 'o, 'r, 'e, '_, 'n, 'b, 'r, y#2894 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -7, -7)) AS naive#2895]
+- Project [family#1920, store_nbr#1922, date#2091, sales#2068 AS y#2894, prediction#2890]
   +- Project [family#1920, store_nbr#1922, date#2091, sales#2068, prediction#2890]
      +- Project [family#1920, store_nbr#1922, date#2091, sales#2068, lag_1#2092, lag_7#2093, lag_14#2094, lag_28#2095, ma_7#2096, ma_28#2098, dow#2100, dom#2101, weekofyear#2102, month#2103, year#2104, family_idx#2847, family_ohe#2855, store_nbr_idx#2865, store_nbr_ohe#2873, features#2886, UDF(features#2886) AS prediction#2890]
         +- Proje

In [31]:
# agg_cols = ['family', 'store_nbr']
agg_cols = ['family']

df_agg = aggregate_to_granularity(
    df=df,
    date_col='date',
    target_col='sales',
    group_cols=agg_cols,
    agg="sum",  # sales are additive
    # extra_numeric_aggs={"price": "mean", "promo_spend": "sum"}  # optional
)

df_feat = build_features(
    df=df_agg,
    date_col='date',
    target_col='sales',
    group_cols=agg_cols,
    lags=[1,2],
    mas=[1,2],
    # add_time_signals=cfg["features"]["add_time_signals"]
)
print(df_feat.count())
df_feat.sort(agg_cols + ["date"]).select(agg_cols + ['date', 'sales', 'lag_1', 'lag_2']).show(10)
df_feat.show(10)

55506
+----------+----------+-----+-----+-----+
|    family|      date|sales|lag_1|lag_2|
+----------+----------+-----+-----+-----+
|AUTOMOTIVE|2013-01-03|161.0|255.0|  0.0|
|AUTOMOTIVE|2013-01-04|169.0|161.0|255.0|
|AUTOMOTIVE|2013-01-05|342.0|169.0|161.0|
|AUTOMOTIVE|2013-01-06|360.0|342.0|169.0|
|AUTOMOTIVE|2013-01-07|189.0|360.0|342.0|
|AUTOMOTIVE|2013-01-08|229.0|189.0|360.0|
|AUTOMOTIVE|2013-01-09|164.0|229.0|189.0|
|AUTOMOTIVE|2013-01-10|164.0|164.0|229.0|
|AUTOMOTIVE|2013-01-11|162.0|164.0|164.0|
|AUTOMOTIVE|2013-01-12|280.0|162.0|164.0|
+----------+----------+-----+-----+-----+
only showing top 10 rows
+----------+----------+-----+-----+-----+-----+-----+---+---+----------+-----+----+
|    family|      date|sales|lag_1|lag_2| ma_1| ma_2|dow|dom|weekofyear|month|year|
+----------+----------+-----+-----+-----+-----+-----+---+---+----------+-----+----+
|AUTOMOTIVE|2013-01-03|161.0|255.0|  0.0|161.0|208.0|  5|  3|         1|    1|2013|
|AUTOMOTIVE|2013-01-04|169.0|161.0|255.0|169.

In [36]:
train, test = train_test_split(df=df_feat, date_col='date', group_cols=agg_cols, mode='horizon', test_horizon=28)
train.show(2)
test.show(2)

+----------+---------+----------+-----+----------+-----+-----+----+----+---+---+----------+-----+----+
|    family|store_nbr|      date|sales|dcoilwtico|lag_1|lag_2|ma_1|ma_2|dow|dom|weekofyear|month|year|
+----------+---------+----------+-----+----------+-----+-----+----+----+---+---+----------+-----+----+
|AUTOMOTIVE|       10|2013-01-03|  2.0|     92.97|  3.0|  0.0| 2.0| 2.5|  5|  3|         1|    1|2013|
|AUTOMOTIVE|       10|2013-01-04|  2.0|     93.12|  2.0|  3.0| 2.0| 2.0|  6|  4|         1|    1|2013|
+----------+---------+----------+-----+----------+-----+-----+----+----+---+---+----------+-----+----+
only showing top 2 rows


+----------+---------+----------+-----+----------+-----+-----+----+----+---+---+----------+-----+----+
|    family|store_nbr|      date|sales|dcoilwtico|lag_1|lag_2|ma_1|ma_2|dow|dom|weekofyear|month|year|
+----------+---------+----------+-----+----------+-----+-----+----+----+---+---+----------+-----+----+
|AUTOMOTIVE|       10|2017-07-19|  0.0|      47.1|  0.0|  0.0| 0.0| 0.0|  4| 19|        29|    7|2017|
|AUTOMOTIVE|       10|2017-07-20|  1.0|     46.73|  0.0|  0.0| 1.0| 0.5|  5| 20|        29|    7|2017|
+----------+---------+----------+-----+----------+-----+-----+----+----+---+---+----------+-----+----+
only showing top 2 rows


In [22]:
agg_cols = ['family', 'family']
# agg_cols = ['family']

df_agg = aggregate_to_granularity(
    df=df,
    date_col='date',
    target_col='sales',
    group_cols=agg_cols,
    agg="sum",  # sales are additive
    # extra_numeric_aggs={"price": "mean", "promo_spend": "sum"}  # optional
)

df_feat = build_features(
    df=df_agg,
    date_col='date',
    target_col='sales',
    group_cols=agg_cols,
    lags=[1,2],
    mas=[1,2],
    # add_time_signals=cfg["features"]["add_time_signals"]
)
print(df_feat.count())
df_feat.sort(agg_cols + ["date"]).select(agg_cols + ['date', 'sales', 'lag_1', 'lag_2']).show(10)

55506
+----------+----------+----------+-----+-----+-----+
|    family|    family|      date|sales|lag_1|lag_2|
+----------+----------+----------+-----+-----+-----+
|AUTOMOTIVE|AUTOMOTIVE|2013-01-03|161.0|255.0|  0.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-04|169.0|161.0|255.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-05|342.0|169.0|161.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-06|360.0|342.0|169.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-07|189.0|360.0|342.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-08|229.0|189.0|360.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-09|164.0|229.0|189.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-10|164.0|164.0|229.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-11|162.0|164.0|164.0|
|AUTOMOTIVE|AUTOMOTIVE|2013-01-12|280.0|162.0|164.0|
+----------+----------+----------+-----+-----+-----+
only showing top 10 rows


In [ ]:
dfx = build_features(df, "date", "sales", ["family", "store_nbr"], lags=[1, 2], mas=[1])
print(dfx.count())
dfx.sort(["family", "store_nbr", "date"]).select(['family', 'store_nbr', 'date', 'sales', 'lag_1', 'lag_2']).show(10)

2997324


+----------+---------+----------+-----+-----+-----+
|    family|store_nbr|      date|sales|lag_1|lag_2|
+----------+---------+----------+-----+-----+-----+
|AUTOMOTIVE|        1|2013-01-03|  3.0|  2.0|  0.0|
|AUTOMOTIVE|        1|2013-01-04|  3.0|  3.0|  2.0|
|AUTOMOTIVE|        1|2013-01-05|  5.0|  3.0|  3.0|
|AUTOMOTIVE|        1|2013-01-06|  2.0|  5.0|  3.0|
|AUTOMOTIVE|        1|2013-01-07|  0.0|  2.0|  5.0|
|AUTOMOTIVE|        1|2013-01-08|  2.0|  0.0|  2.0|
|AUTOMOTIVE|        1|2013-01-09|  2.0|  2.0|  0.0|
|AUTOMOTIVE|        1|2013-01-10|  2.0|  2.0|  2.0|
|AUTOMOTIVE|        1|2013-01-11|  3.0|  2.0|  2.0|
|AUTOMOTIVE|        1|2013-01-12|  2.0|  3.0|  2.0|
+----------+---------+----------+-----+-----+-----+
only showing top 10 rows


In [23]:
dfx = build_features(df, "date", "sales", ["family"], lags=[1, 2], mas=[1])
print(dfx.count())
dfx.sort(["family", "store_nbr", "date"]).select(['family', 'store_nbr', 'date', 'sales', 'lag_1', 'lag_2']).show(10)

3000822


+----------+---------+----------+-----+-----+-----+
|    family|store_nbr|      date|sales|lag_1|lag_2|
+----------+---------+----------+-----+-----+-----+
|AUTOMOTIVE|        1|2013-01-01|  0.0|  0.0|  0.0|
|AUTOMOTIVE|        1|2013-01-02|  2.0| 11.0|  0.0|
|AUTOMOTIVE|        1|2013-01-03|  3.0|  0.0|  5.0|
|AUTOMOTIVE|        1|2013-01-04|  3.0|  0.0|  1.0|
|AUTOMOTIVE|        1|2013-01-05|  5.0|  0.0|  6.0|
|AUTOMOTIVE|        1|2013-01-06|  2.0|  7.0|  5.0|
|AUTOMOTIVE|        1|2013-01-07|  0.0|  5.0|  7.0|
|AUTOMOTIVE|        1|2013-01-08|  2.0|  2.0| 14.0|
|AUTOMOTIVE|        1|2013-01-09|  2.0|  7.0|  0.0|
|AUTOMOTIVE|        1|2013-01-10|  2.0| 12.0|  3.0|
+----------+---------+----------+-----+-----+-----+
only showing top 10 rows


In [ ]:
# Load config from YAML or dict
import yaml, json
cfg = yaml.safe_load(open("forecast_config.yaml"))

out = run_forecast(df_feat, cfg)

display(out["predictions"])         # per-group predictions on test window
# display(out["metrics_portfolio"])   # wMAPE, sMAPE, MASE overall
# display(out["metrics_by_series"])   # same metrics by series
# display(out["backtest"])            # rolling-origin backtest summary (if enabled)
